In [1]:
import pandas as pd
import spacy
import sddk
import requests

In [2]:
nlp = spacy.load('la_core_web_lg')

In [5]:
s = sddk.cloudSession(provider="sciencedata.dk", shared_folder_name="TOME/DATA/NOSCEMUS", owner="kase@zcu.cz")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/DATA/NOSCEMUS/


In [6]:
dir_ids_list = s.list_directories("NOSCEMUS_FULL/")
dir_ids_list[:10]

['1031760',
 '1085290',
 '1285853',
 '1285854',
 '1285855',
 '1285856',
 '1365811',
 '1370560',
 '1378359',
 '1424044']

In [7]:
len(dir_ids_list)

1009

In [8]:
for id in dir_ids_list[:10]:
    for textfile in s.list_filenames("NOSCEMUS_FULL/" + id, "txt"):
        print(id, textfile)

1031760 Bacon,_Francis_-_Instauratio_magna__London_1620_pdf.txt
1085290 Linden,_Johannes_Antonides_van_der_-_Lindenius_renovatus__Nuremberg_1686_pdf.txt
1285853 de_Conde,_Ioannes_Baptista_-_Aphorismi_seu_axiomata_Hippocratis_versu_heroico_explicata__Brussels_1647_pdf.txt
1285854 van_Poort,_Henricus_-_Hippocratis_Aphorismi_metrica_praphrasi_translati__Utrecht_1657_pdf.txt
1285855 Hippocrates_&_Denisot,_Gérard_-_Hippocratis_Aphorismis_versibus_Graecis_et_Latini_expositi__Paris_1634_pdf.txt
1285856 Hippocrates_&_Berigardus,_Petrus_-_Hippocratis_Aphorismi_rhytmici__Udine_1645_pdf.txt
1365811 Acta_literaria_Sueciae__Vol__1__Uppsala_[1723–1725]_.txt
1370560 Quensel,_Conrad_&_Eurodius,_Hans_-_De_lumine_nocturno_Boreali__Lund_1726.txt
1378359 Celsius,_Anders_-_Observationes_de_lumine_Boreali__Nuremberg_1733.txt
1424044 Bacci,_Andrea_&_Gabelkover,_Wolfgang_-_De_monocerote_seu_unicornu__Stuttgart_1598_pdf.txt


In [18]:
filenames_ids_dict = {}
missing_ids = []
for id in dir_ids_list[:10]:
    try:
        filenames = s.list_filenames("NOSCEMUS_FULL/" + id, "txt")
        if len(filenames) > 1:
            n = 1
            for filename in filenames:
                filenames_ids_dict[filename] = id
                text = s.read_file("NOSCEMUS_FULL/" + id + "/" + filename, "str")
                s.write_file("noscemus_raw/{0}_{1}.txt".format(str(id), str(n)), text)
                n += 1
        else:
            filename = filenames[0] 
            filenames_ids_dict[filename] = id
            text = s.read_file("NOSCEMUS_FULL/" + id + "/" + filename, "str")
            s.write_file("noscemus_raw/{0}.txt".format(str(id)), text)
    except:
        missing_ids.append(id)